# TSA Claims data
With these claims we'll try to answer questions like: What are people complaining about to TSA, how much are they claiming, which airlines are paying out. We'll accomplish this by building horizontal column charts, x axis is dollar amount, y axis are categories and bins of columns are years.

<strong>Uses Python 3 and Bokeh 13</strong>

* Datasource: https://www.dhs.gov/tsa-claims-data
* Chart inspiration: https://bokeh.pydata.org/en/latest/docs/gallery/bar_stacked_split.html

This notebook does not use all the available TSA claims data. The latest was only available in PDF form nobody got time for that.

#### www.machineloveus.com

In [1]:
import pandas as pd

In [2]:
sheets = ['claims-2002-2006_0.xls', 'claims-2007-2009_0.xls',
          'claims-2010-2013_0.xls', 'claims-2014.xls', 'claims-data-2015-as-of-feb-9-2016.xls']
frames = []
for sheet in sheets:
    df = pd.read_excel(sheet)
    df = df[(df['Disposition'] == 'Approve in Full') |
        (df['Disposition'] == 'Settle')]
    frames.append(df)
df = pd.concat(frames)

/Users/cenchen/repos/prolific/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [3]:
df.shape

(81107, 15)

In [4]:
df.head(n=3)

,Airline Name,Airport Code,Airport Name,Claim Amount,Claim Number,Claim Site,Claim Type,Close Amount,Date Received,Disposition,Incident D,Incident Date,Item,Item Category,Status
0,Continental Airlines,EWR,Newark International Airport,350.00,0909802M,Checkpoint,Property Damage,350,2002-01-04 00:00:00,Approve in Full,NaT,2002-12-12 00:00:00,Other,NaN,Approved
1,NaN,SEA,Seattle-Tacoma International,100.00,0202417M,Checked Baggage,Property Damage,50,2002-02-02 00:00:00,Settle,NaT,2004-01-16 00:00:00,Luggage (all types including footlockers),NaN,Settled
2,American Airlines,STL,Lambert St. Louis International,278.88,0202445M,Checked Baggage,Property Damage,227.92,2002-02-04 00:00:00,Settle,NaT,2003-11-26 00:00:00,Cell Phones,NaN,Settled


In [5]:
df['Claim Type'].unique()

array(['Property Damage', 'Passenger Property Loss', nan, 'Motor Vehicle',
       'Personal Injury', 'Passenger Theft', 'Employee Loss (MPCECA)',
       '-', 'Bus Terminal', 'Complaint'], dtype=object)

In [6]:
df['Disposition'].unique()

array(['Approve in Full', 'Settle'], dtype=object)

In [7]:
df.shape

(81107, 15)

In [8]:
#df.Item.unique() #Bags, books, clothes and jewelry, electronics, other

In [9]:
df['items'] = df['Item'].astype(str) + df['Item Category'].astype(str)

In [10]:
df = df[['items', 'Date Received', 'Close Amount']]
df = df.dropna(axis='rows')

In [11]:
bags = ['bags', 'bag', 'luggage', "accessories"]
books = ['books', 'book']
clothes_and_jewlery = ['clothing', 'clothes', 'jewelry', 'watches']
electronics = ['mp3', 'ipods', 'stereo', 'camera', 'players', 
               'cameras', 'phones', 'phone', 'computer']

for i, row in df.iterrows():
    lowered = row['items'].lower()
    category = 'Other'

    for j in bags:
        if j in lowered:
            category = 'Bags and luggage'
            break
            
    for k in books:
        if k in lowered and category == 'Other':
            category = 'Books'
            break
    
    for l in clothes_and_jewlery:
        if l in lowered and category == 'Other':
            category = 'Clothes and jewlery'
            break

    for m in electronics:
        if m in lowered and category == 'Other':
            category = 'Electronics of various kinds'
            break
    
    df.set_value(i,  'category', category)

/Users/cenchen/repos/prolific/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [12]:
df.category.unique()

array(['Other', 'Clothes and jewlery', 'Bags and luggage',
       'Electronics of various kinds', 'Books'], dtype=object)

In [13]:
for i, row in df.iterrows():
    year = None
    try:
        year = int(row['Date Received'].year)
    except Exception:
        print(row['Date Received'])
        year = -1
    
    df.set_value(i,  'year', year)

/Users/cenchen/repos/prolific/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':


In [14]:
df.shape

(80889, 5)

In [15]:
y_range = list(df.category.unique())

In [16]:
y_range

['Other',
 'Clothes and jewlery',
 'Bags and luggage',
 'Electronics of various kinds',
 'Books']

In [17]:
df.year.unique()

array([2002., 2015., 2007., 2014., 2010., 2003., 2011., 2008., 2012.,
       2004., 2013., 2009., 2005., 2006., 2020., 2025., 2040., 2094.,
       2204., 2205., 2206., 2207., 2500., 3005.])

In [18]:
df = df[df['year']>0]
REAL_YEARS = [2002, 2003, 2004, 2005, 2006, 2007, 2008,
              2009, 2010, 2011, 2012, 2013, 2014, 2015]

In [19]:
# Group by year

In [20]:
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Inferno, Viridis, d3 #GnBu10, OrRd5
from bokeh.plotting import figure


In [21]:
output_file("bar_stacked_split.html")

In [22]:
plotting = df.groupby(['year', 'category'], as_index=False).agg({'Close Amount': sum})

In [23]:
plotting.head()

,year,category,Close Amount
0,2002.0,Bags and luggage,9855.91
1,2002.0,Clothes and jewlery,3606.45
2,2002.0,Electronics of various kinds,10916.81
3,2002.0,Other,12876.22
4,2003.0,Bags and luggage,235949.06


In [24]:
dct = {'Other': 0,
       'Electronics of various kinds': 0,
       'Bags and luggage': 0,
       'Clothes and jewlery': 0,
       'Books': 0}

In [25]:
data = []
for yr in REAL_YEARS:
    for k, v in dct.items():
        data.append({'year': yr, 'category': k,
                     'Close Amount': v})
default_df = pd.DataFrame(data)

In [26]:
plotting = plotting.merge(default_df, how='outer', on=['year', 'category'])

In [27]:
for i, r in plotting.iterrows():
    amt = 0
    if r['Close Amount_x'] > 0:
        amt = r['Close Amount_x']
    plotting.set_value(i,  'amount', amt)

/Users/cenchen/repos/prolific/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """


In [28]:
categories = ['Bags and luggage', 'Books', 'Clothes and jewlery',
              'Electronics of various kinds', 'Other']
sub_categories = REAL_YEARS
data = {'Categories': categories}
for sub in sub_categories:
    data[str(sub)] = []
    
data

{'Categories': ['Bags and luggage',
  'Books',
  'Clothes and jewlery',
  'Electronics of various kinds',
  'Other'],
 '2002': [],
 '2003': [],
 '2004': [],
 '2005': [],
 '2006': [],
 '2007': [],
 '2008': [],
 '2009': [],
 '2010': [],
 '2011': [],
 '2012': [],
 '2013': [],
 '2014': [],
 '2015': []}

In [29]:
plotting = plotting.sort_values(by=['category', 'year'])

In [30]:
plotting.head(n=3)

,year,category,Close Amount_x,Close Amount_y,amount
0,2002.0,Bags and luggage,9855.91,0.0,9855.91
4,2003.0,Bags and luggage,235949.06,0.0,235949.06
8,2004.0,Bags and luggage,360740.95,0.0,360740.95


In [31]:
for i, r in plotting.iterrows():
    if r['year'] in REAL_YEARS:
        good_year = str(int(r['year']))
        data[good_year].append(r['amount'])

In [32]:
data

{'Categories': ['Bags and luggage',
  'Books',
  'Clothes and jewlery',
  'Electronics of various kinds',
  'Other'],
 '2002': [9855.91, 0.0, 3606.45, 10916.81, 12876.22],
 '2003': [235949.06000000003, 0.0, 180988.53, 250283.7, 468978.6400000006],
 '2004': [360740.9500000002,
  0.0,
  384068.69999999966,
  532669.8699999994,
  915037.0799999945],
 '2005': [346881.77999999974,
  0.0,
  269525.52999999997,
  553951.1799999992,
  892788.3099999935],
 '2006': [245658.81999999998,
  2301.5900000000006,
  204396.5300000001,
  473016.39999999915,
  582581.1999999982],
 '2007': [158743.91000000003,
  1344.16,
  95295.23000000003,
  216112.1,
  321113.07999999996],
 '2008': [148335.69000000012,
  1758.1000000000001,
  78736.92,
  231933.1699999998,
  320657.90999999986],
 '2009': [132620.67,
  2219.9900000000007,
  74024.42,
  202965.39,
  476225.15999999916],
 '2010': [251933.38999999993,
  1696.88,
  104355.7600000001,
  22428.3,
  208771.51000000015],
 '2011': [588923.749999999,
  2751.76999

In [33]:
subs = []
for i in REAL_YEARS:
    subs.append(str(i))

In [34]:
c14 =  d3['Category20c'][14] #Inferno[14] #Viridis[10]
c13 =  d3['Category20c'][13] #Inferno[14] #Viridis[10]

In [35]:
from bokeh.models import NumeralTickFormatter
p = figure(y_range=y_range, x_range=[0, 8000000],
           plot_height=600,
           title="Approved or settled claims by category, by year",
           toolbar_location=None)

p.hbar_stack(subs, y='Categories',
             height=0.9,
             color=c14,
             source=ColumnDataSource(data),
             legend=["%s " % x for x in subs])

p.y_range.range_padding = 0.1
p.ygrid.grid_line_color = None
p.legend.location = "top_right"
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.xaxis.formatter=NumeralTickFormatter(format="0.0a")

top = p

In [36]:
#show(p)

## Second visualization
Which airlines are paying out the most/least

In [37]:
df = pd.concat(frames)

/Users/cenchen/repos/prolific/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [38]:
df = df[['Airport Code', 'Date Received', 'Close Amount']]
df = df.dropna(axis='rows')

In [39]:
airports = list(df['Airport Code'].unique())

In [40]:
for i, row in df.iterrows():
    year = None
    try:
        year = str(int(row['Date Received'].year))
    except Exception:
        print(row['Date Received'])
        year = -1
    
    df.set_value(i,  'year', year)

/Users/cenchen/repos/prolific/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':


In [41]:
plotting = df.groupby(['year', 'Airport Code'], as_index=False).agg({'Close Amount': sum})

In [42]:
grp = df.groupby(['Airport Code'], as_index=False).agg({'Close Amount': sum})

In [43]:
plotting.head()

,year,Airport Code,Close Amount
0,2002,ATL,1022.0
1,2002,AUS,23.0
2,2002,BDL,480.0
3,2002,BHM,20.0
4,2002,BNA,110.0


In [44]:
grp['total'] = grp['Close Amount']
del grp['Close Amount']

In [45]:
grp.head()

,Airport Code,total
0,ABE,9523.94
1,ABI,2297.68
2,ABQ,66639.88
3,ABR,1347.01
4,ABY,815.09


In [46]:
plotting = plotting.merge(grp, on='Airport Code')

In [47]:
plotting.describe()

,Close Amount,total
count,3239.000000,3239.000000
mean,4854.459759,66907.876817
std,11638.169300,135905.924459
min,5.000000,6.350000
25%,155.560000,2685.090000
50%,641.060000,10510.150000
75%,3587.385000,55221.370000
max,140181.980000,965646.980000


In [48]:
plotting.total.quantile(.95)

345437.3999999993

In [49]:
plotting = plotting[plotting['total']>=plotting.total.quantile(.95)]

In [50]:
plotting.shape

(174, 4)

In [51]:
airports = list(plotting['Airport Code'].unique())
airports = sorted(airports)
years = list(plotting['year'].unique())

In [52]:
print(airports)

['ATL', 'BOS', 'DFW', 'EWR', 'JFK', 'LAX', 'LGA', 'MCO', 'MIA', 'ORD', 'PHX', 'SEA']


In [53]:
len(airports)

12

In [54]:
STRING_YEARS = [str(r) for r in REAL_YEARS]

In [55]:
data = []
for yr in STRING_YEARS:
    for airport in airports:
        data.append({'year': yr, 'Airport Code': airport,
                     'Close Amount': 0})
default_df = pd.DataFrame(data)

In [56]:
plotting = plotting.merge(default_df, how='outer', on=['year', 'Airport Code'])

In [57]:
plotting['amount'] = plotting['Close Amount_x'] + plotting['Close Amount_y']

In [58]:
plotting.head()

,year,Airport Code,Close Amount_x,total,Close Amount_y,amount
0,2002,ATL,1022.00,471122.13,0.0,1022.00
1,2003,ATL,19911.30,471122.13,0.0,19911.30
2,2004,ATL,52128.54,471122.13,0.0,52128.54
3,2005,ATL,61882.41,471122.13,0.0,61882.41
4,2006,ATL,44161.03,471122.13,0.0,44161.03


In [59]:
plotting.year.unique()

array(['2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2040', '2500',
       '2020', '2094', '2025', '2207'], dtype=object)

In [60]:
sub_categories = STRING_YEARS
data = {'Categories': airports}
for sub in sub_categories:
    data[sub] = []

In [61]:
plotting = plotting.sort_values(by=['Airport Code', 'year'])

In [62]:
plotting.head(n=3)

,year,Airport Code,Close Amount_x,total,Close Amount_y,amount
0,2002,ATL,1022.00,471122.13,0.0,1022.00
1,2003,ATL,19911.30,471122.13,0.0,19911.30
2,2004,ATL,52128.54,471122.13,0.0,52128.54


In [63]:
data

{'Categories': ['ATL',
  'BOS',
  'DFW',
  'EWR',
  'JFK',
  'LAX',
  'LGA',
  'MCO',
  'MIA',
  'ORD',
  'PHX',
  'SEA'],
 '2002': [],
 '2003': [],
 '2004': [],
 '2005': [],
 '2006': [],
 '2007': [],
 '2008': [],
 '2009': [],
 '2010': [],
 '2011': [],
 '2012': [],
 '2013': [],
 '2014': [],
 '2015': []}

In [64]:
for i, r in plotting.iterrows():
    if r['year'] in STRING_YEARS:
        data[r['year']].append(r['amount'])
    else:
        print(r)

year                2040
Airport Code         ATL
Close Amount_x      8.48
total             471122
Close Amount_y       NaN
amount               NaN
Name: 14, dtype: object
year                2500
Airport Code         EWR
Close Amount_x      35.7
total             667754
Close Amount_y       NaN
amount               NaN
Name: 57, dtype: object
year                2020
Airport Code         JFK
Close Amount_x    179.08
total             777054
Close Amount_y       NaN
amount               NaN
Name: 72, dtype: object
year                2094
Airport Code         JFK
Close Amount_x        50
total             777054
Close Amount_y       NaN
amount               NaN
Name: 73, dtype: object
year                2025
Airport Code         MCO
Close Amount_x       100
total             484125
Close Amount_y       NaN
amount               NaN
Name: 116, dtype: object
year                2207
Airport Code         MIA
Close Amount_x    162.99
total             475041
Close Amount_y       NaN
amou

In [65]:
colors = d3['Category20c'][len(STRING_YEARS)]

In [66]:
airports

['ATL',
 'BOS',
 'DFW',
 'EWR',
 'JFK',
 'LAX',
 'LGA',
 'MCO',
 'MIA',
 'ORD',
 'PHX',
 'SEA']

In [67]:
data.keys()

dict_keys(['Categories', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015'])

In [68]:
STRING_YEARS

['2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015']

In [69]:
len(colors) == len(STRING_YEARS)

True

In [70]:
airports == data['Categories']


True

In [71]:
len(STRING_YEARS) == len(data)-1

True

In [72]:
from bokeh.models import NumeralTickFormatter
p = figure(y_range=airports, x_range=[0, 1000000],
           plot_height=800,
           title="Approved or settled claims by airport, by year",
           toolbar_location=None)

p.hbar_stack(STRING_YEARS, y='Categories',
             height=0.9,
             color=colors,
             source=ColumnDataSource(data),
             legend=["%s " % yr for yr in STRING_YEARS])

p.y_range.range_padding = 0.1
p.ygrid.grid_line_color = None
p.legend.location = "top_right"
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.xaxis.formatter=NumeralTickFormatter(format="0.0a")

bottom = p

In [73]:
from bokeh.io import output_file, show
from bokeh.layouts import column
from bokeh.plotting import figure

In [74]:
output_file("layout_tsa.html")

In [75]:
show(column(top, bottom))

In [ ]:
# Thanks for reading, visit www.machineloveus.com for more data science content.